In [ ]:
import pandas as pd
%matplotlib inline

from sklearn.model_selection import train_test_split

from model import optimize_function,optimize,optimize_stacking

from sklearn.linear_model import LogisticRegression # ロジスティック回帰
from sklearn.neighbors import KNeighborsClassifier # K近傍法
from sklearn.svm import SVC # サポートベクターマシン
from sklearn.tree import DecisionTreeClassifier # 決定木
from sklearn.ensemble import RandomForestClassifier # ランダムフォレスト
from lightgbm import LGBMClassifier #LGBM
from sklearn.ensemble import AdaBoostClassifier # AdaBoost
from sklearn.naive_bayes import GaussianNB # ナイーブ・ベイズ

from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score

データの読み込みと分割

In [ ]:
# データの読み込み
data = pd.read_csv('../../data/preprocessed_data.csv',index_col=None)

# 特徴量とターゲット変数の分割
X = data.drop('dengue', axis=1)  
y = data['dengue']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

各分類機の最適化

In [ ]:
optimize_function=optimize_function(X_train,y_train)

In [ ]:
objectives_dict={"Logistic Regression" : optimize_function.objective_logistic_regression,
                 "Nearest Neighbors" : optimize_function.objective_knn, 
                 "RBF SVM" : optimize_function.objective_svm_rbf, 
                 "Decision Tree" : optimize_function.objective_decision_tree,
                 "Random Forest" : optimize_function.objective_random_forest,
                 "LGBM" : optimize_function.objective_lgbm,
                 "AdaBoost" : optimize_function.objective_adaboost, 
                 "Naive Bayes": optimize_function.objective_naive_bayes
                 }

In [ ]:
optimize=optimize(objectives_dict)
best_params=optimize.optimize()

In [ ]:
df_best_params=pd.DataFrame.from_dict(best_params,orient='index')
df_best_params.to_csv('best_params/best_params.csv')

In [ ]:
best_params_df=pd.read_csv('best_params/best_params.csv')
best_params_df

スタッキングモデルの構築

In [ ]:
names = ['RBF SVM','Nearest Neighbors', 
         'Decision Tree','Random Forest','AdaBoost']

classifiers = [SVC(kernel="rbf",C=0.35,gamma='scale',class_weight='balanced',probability=True),
               KNeighborsClassifier(n_neighbors=38,weights='uniform',algorithm='kd_tree'),
               DecisionTreeClassifier(criterion='entropy',max_depth=25,min_samples_split=2,min_samples_leaf=100,class_weight='balanced'),
               RandomForestClassifier(max_depth=84,min_samples_split=23,min_samples_leaf=49,n_estimators=146,class_weight='balanced'),
               AdaBoostClassifier(algorithm='SAMME',n_estimators=867,learning_rate=0.575176)]

In [ ]:
optimize_stacking=optimize_stacking(names,classifiers,X_train,y_train)
objective_stacking=optimize_stacking.objective_stacking
optimize_stacking.optimize_stacking(objective_stacking)

In [ ]:
# 最適化したスタッキングモデルの構築
#meta_model = LGBMClassifier(num_leaves=611,learning_rate=0.073,n_estimators=897,lambda_l1=15,lambd_l2=39) #LGBM
meta_model=LogisticRegression()
estimator=[(name,clf) for name,clf in zip(names,classifiers)]
stacking_clf = StackingClassifier(estimators=estimator, final_estimator=meta_model)
stacking_clf.fit(X_train, y_train)


In [ ]:
#テスト・比較
result = []

#各機械学習モデルのテスト
for name, clf in zip(names, classifiers): 
    clf.fit(X_train, y_train) 
    score1 = clf.score(X_train, y_train) 
    score2 = clf.score(X_test, y_test) 
    result.append([score1, score2]) 

result = pd.DataFrame(result, columns=['train', 'test'], index=names)

#スタッキングモデルのテスト
y_pred = stacking_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

stacking_score1 = stacking_clf.score(X_train, y_train)
stacking_score2 = stacking_clf.score(X_test, y_test)

stacking_result = pd.DataFrame([[stacking_score1, stacking_score2]], columns=['train', 'test'], index=['Stacking'])

# 既存の結果とスタッキング結果を結合
result_df = pd.concat([pd.DataFrame(result, columns=['train', 'test'], index=names), stacking_result])
result_df = result_df.sort_values('test', ascending=False)

print(result_df)


In [ ]:
result_df.plot(kind='bar', alpha=0.5, grid=True)